In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests11 as vuong_tests_fast

In [2]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y


def setup_shi(yn,xn,return_model=False,num_params=4):
    x1n,x2n = xn[:,0],xn[:,1:num_params+1]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    model1_deriv = OLS_loglike(yn,sm.add_constant(x1n))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    model2_deriv = OLS_loglike(yn,sm.add_constant(x2n))
    ll2 = model2_deriv.loglikeobs(model2_fit.params)
    grad2 =  model2_deriv.score_obs(model2_fit.params)    
    hess2 = model2_deriv.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2


In [3]:
def gen_data(nobs=1000, a=0.25, num_params=4):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x[:,0] + a/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn,return_model=False,num_params=15)
print(grad1.shape,hess1.shape)

(1000, 2) (2, 2)


In [4]:
num_sims = 500
trials =1000
adapt_c = False
data_tuned_epsilon = True
epsilon = .5

size weirdness rn...

In [5]:
a = 0.25
num_params=9

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.96 & 0.96 & 0.96 & 0.82 & 0.95 & 0.97   \\
Model 1 & 0.01 & 0.01 & 0.02 & 0.11 & 0.03 & 0.02   \\
Model 2 & 0.04 & 0.04 & 0.02 & 0.07 & 0.02 & 0.01   \\
\hline
\end{tabular}
(array([0.956, 0.008, 0.036]), array([0.956, 0.008, 0.036]), array([0.96 , 0.018, 0.022]), array([0.816, 0.11 , 0.074]), array([0.946, 0.036, 0.018]), array([0.96 , 0.018, 0.022]), array([0.95, 0.03, 0.02]), array([0.97, 0.02, 0.01]), -3.3357660689797326, 10.72993948109056, 10.97779084353534)


# power 0.... need to see...

In [6]:
a1,a2 = np.sqrt(1.09-1), 0.00
num_params= 9


def gen_data2(nobs=1000, a1=np.sqrt(1.09-1), a2=0.00 , num_params=19):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a1*x[:,0] + a2/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

In [7]:
calc_c =lambda nobs: (nobs**(-1/2)/100,nobs**(1/5) )

nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.81 & 0.82 & 0.70 & 0.49 & 0.83 & 0.55   \\
Model 1 & 0.19 & 0.18 & 0.30 & 0.51 & 0.17 & 0.45   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.814, 0.186, 0.   ]), array([0.816, 0.184, 0.   ]), array([0.7, 0.3, 0. ]), array([0.494, 0.506, 0.   ]), array([0.492, 0.508, 0.   ]), array([0.7, 0.3, 0. ]), array([0.83, 0.17, 0.  ]), array([0.554, 0.446, 0.   ]), 6.551308383801216, 5.284270944614708, 5.328898589783983)


In [8]:
nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.84 & 0.84 & 0.78 & 0.54 & 0.86 & 0.58   \\
Model 1 & 0.16 & 0.16 & 0.22 & 0.45 & 0.13 & 0.42   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.844, 0.156, 0.   ]), array([0.844, 0.156, 0.   ]), array([0.778, 0.222, 0.   ]), array([0.542, 0.454, 0.004]), array([0.558, 0.442, 0.   ]), array([0.778, 0.222, 0.   ]), array([0.864, 0.134, 0.002]), array([0.58, 0.42, 0.  ]), 6.417165348116173, 5.364744563547969, 5.385940423523317)


In [9]:
nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.82 & 0.83 & 0.77 & 0.52 & 0.83 & 0.55   \\
Model 1 & 0.18 & 0.17 & 0.23 & 0.48 & 0.17 & 0.45   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.822, 0.178, 0.   ]), array([0.828, 0.172, 0.   ]), array([0.774, 0.226, 0.   ]), array([0.522, 0.478, 0.   ]), array([0.524, 0.476, 0.   ]), array([0.774, 0.226, 0.   ]), array([0.83, 0.17, 0.  ]), array([0.552, 0.448, 0.   ]), 6.404668422107189, 4.991720117253817, 5.3201759558740065)


In [10]:
nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.84 & 0.85 & 0.73 & 0.51 & 0.82 & 0.59   \\
Model 1 & 0.16 & 0.15 & 0.27 & 0.48 & 0.17 & 0.41   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.01 & 0.01 & 0.00   \\
\hline
\end{tabular}
(array([0.842, 0.158, 0.   ]), array([0.846, 0.154, 0.   ]), array([0.73, 0.27, 0.  ]), array([0.514, 0.478, 0.008]), array([0.524, 0.476, 0.   ]), array([0.73, 0.27, 0.  ]), array([0.824, 0.17 , 0.006]), array([0.592, 0.408, 0.   ]), 6.458393346904825, 5.083620602698635, 5.3937783014112695)


In [11]:
data_tuned_c = .9
calc_c =lambda nobs: (nobs**(-1/2)/50,nobs**(1/10) )

nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.82 & 0.83 & 0.71 & 0.52 & 0.83 & 0.57   \\
Model 1 & 0.18 & 0.17 & 0.29 & 0.48 & 0.17 & 0.43   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.01 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.822, 0.178, 0.   ]), array([0.828, 0.172, 0.   ]), array([0.706, 0.294, 0.   ]), array([0.518, 0.476, 0.006]), array([0.554, 0.446, 0.   ]), array([0.706, 0.294, 0.   ]), array([0.826, 0.174, 0.   ]), array([0.568, 0.432, 0.   ]), 6.573691192646283, 5.179427307833351, 5.346159648358514)


# a  = .25, k= 9

In [12]:
a = 0.25
num_params=9



In [13]:
nobs = 250

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.90 & 0.90 & 0.95 & 0.84 & 0.94 & 0.98   \\
Model 1 & 0.01 & 0.01 & 0.01 & 0.09 & 0.02 & 0.02   \\
Model 2 & 0.09 & 0.09 & 0.04 & 0.07 & 0.04 & 0.01   \\
\hline
\end{tabular}
(array([0.898, 0.008, 0.094]), array([0.9  , 0.008, 0.092]), array([0.95 , 0.014, 0.036]), array([0.844, 0.09 , 0.066]), array([0.944, 0.04 , 0.016]), array([0.95 , 0.014, 0.036]), array([0.936, 0.024, 0.04 ]), array([0.976, 0.016, 0.008]), -4.522820683800228, 5.8655256587836755, 6.0396203282992635)


In [14]:
nobs = 500

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.93 & 0.93 & 0.95 & 0.83 & 0.95 & 0.97   \\
Model 1 & 0.00 & 0.00 & 0.02 & 0.12 & 0.03 & 0.01   \\
Model 2 & 0.07 & 0.07 & 0.03 & 0.05 & 0.02 & 0.02   \\
\hline
\end{tabular}
(array([0.926, 0.002, 0.072]), array([0.926, 0.002, 0.072]), array([0.95 , 0.024, 0.026]), array([0.834, 0.12 , 0.046]), array([0.946, 0.042, 0.012]), array([0.95 , 0.024, 0.026]), array([0.95 , 0.032, 0.018]), array([0.97 , 0.012, 0.018]), -4.155776956178211, 7.846597091126985, 8.059889407995662)


In [15]:
nobs = 100

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.88 & 0.95 & 0.96 & 0.85 & 0.92 & 0.99   \\
Model 1 & 0.00 & 0.00 & 0.01 & 0.12 & 0.03 & 0.00   \\
Model 2 & 0.12 & 0.05 & 0.02 & 0.04 & 0.05 & 0.01   \\
\hline
\end{tabular}
(array([0.876, 0.   , 0.124]), array([0.95, 0.  , 0.05]), array([0.964, 0.014, 0.022]), array([0.846, 0.118, 0.036]), array([0.926, 0.066, 0.008]), array([0.964, 0.014, 0.022]), array([0.924, 0.03 , 0.046]), array([0.99 , 0.004, 0.006]), -4.222134295837247, 4.107289051829694, 4.37170178126934)


# a  = .25, k= 4, something not right?

In [16]:
a = 0.25
num_params=4



In [17]:
nobs = 250

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.94 & 0.94 & 0.95 & 0.82 & 0.93 & 0.98   \\
Model 1 & 0.00 & 0.00 & 0.02 & 0.10 & 0.04 & 0.01   \\
Model 2 & 0.05 & 0.05 & 0.03 & 0.07 & 0.03 & 0.01   \\
\hline
\end{tabular}
(array([0.944, 0.004, 0.052]), array([0.944, 0.004, 0.052]), array([0.95 , 0.024, 0.026]), array([0.824, 0.102, 0.074]), array([0.936, 0.042, 0.022]), array([0.95 , 0.024, 0.026]), array([0.926, 0.044, 0.03 ]), array([0.982, 0.006, 0.012]), -1.6949358928683036, 5.859968416769919, 5.654504197358035)


# a  = .25, K = 19

In [18]:
a = 0.25
num_params=19


In [19]:
nobs = 250

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.70 & 0.81 & 0.94 & 0.80 & 0.93 & 0.96   \\
Model 1 & 0.00 & 0.00 & 0.01 & 0.17 & 0.02 & 0.02   \\
Model 2 & 0.30 & 0.19 & 0.05 & 0.04 & 0.05 & 0.02   \\
\hline
\end{tabular}
(array([0.702, 0.   , 0.298]), array([0.812, 0.   , 0.188]), array([0.944, 0.01 , 0.046]), array([0.796, 0.166, 0.038]), array([0.9  , 0.096, 0.004]), array([0.944, 0.01 , 0.046]), array([0.928, 0.024, 0.048]), array([0.964, 0.018, 0.018]), -9.536300252858078, 6.446764322507757, 6.676671753278817)


# evidence of power

In [20]:
nobs = 500

a1,a2 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.98 & 0.98 & 0.86 & 0.57 & 0.90 & 0.65   \\
Model 1 & 0.01 & 0.01 & 0.14 & 0.42 & 0.10 & 0.35   \\
Model 2 & 0.01 & 0.01 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.978, 0.014, 0.008]), array([0.98 , 0.014, 0.006]), array([0.862, 0.138, 0.   ]), array([0.574, 0.422, 0.004]), array([0.606, 0.394, 0.   ]), array([0.862, 0.138, 0.   ]), array([0.9  , 0.096, 0.004]), array([0.652, 0.348, 0.   ]), 1.3686975916070712, 5.841402280086362, 6.267731223132327)


In [21]:
nobs = 100

a1,a2 = np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.96 & 1.00 & 0.78 & 0.36 & 0.88 & 0.68   \\
Model 1 & 0.02 & 0.00 & 0.22 & 0.64 & 0.12 & 0.32   \\
Model 2 & 0.02 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.962, 0.02 , 0.018]), array([0.996, 0.   , 0.004]), array([0.78, 0.22, 0.  ]), array([0.362, 0.638, 0.   ]), array([0.34, 0.66, 0.  ]), array([0.78, 0.22, 0.  ]), array([0.876, 0.12 , 0.004]), array([0.678, 0.322, 0.   ]), 0.5101461501477029, 5.352770815603537, 5.783720707058467)


In [22]:
nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=4)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.56 & 0.56 & 0.64 & 0.54 & 0.79 & 0.51   \\
Model 1 & 0.44 & 0.44 & 0.36 & 0.46 & 0.21 & 0.49   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.556, 0.444, 0.   ]), array([0.558, 0.442, 0.   ]), array([0.638, 0.36 , 0.002]), array([0.54 , 0.456, 0.004]), array([0.578, 0.42 , 0.002]), array([0.638, 0.36 , 0.002]), array([0.79 , 0.208, 0.002]), array([0.514, 0.486, 0.   ]), 9.56098274898735, 4.699087714435717, 4.948435310494212)


In [23]:
nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=19)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.97 & 0.98 & 0.82 & 0.45 & 0.88 & 0.65   \\
Model 1 & 0.02 & 0.02 & 0.18 & 0.55 & 0.12 & 0.35   \\
Model 2 & 0.01 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.97 , 0.024, 0.006]), array([0.976, 0.022, 0.002]), array([0.82, 0.18, 0.  ]), array([0.454, 0.546, 0.   ]), array([0.426, 0.574, 0.   ]), array([0.82, 0.18, 0.  ]), array([0.88, 0.12, 0.  ]), array([0.652, 0.348, 0.   ]), 1.6303682748259436, 5.644345012807255, 6.168883015930556)


# evidence of power 2

In [24]:
nobs = 250


a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.02 & 0.56 & 0.60 & 0.79 & 0.83 & 0.65   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.03 & 0.00 & 0.00   \\
Model 2 & 0.98 & 0.44 & 0.40 & 0.18 & 0.17 & 0.35   \\
\hline
\end{tabular}
(array([0.018, 0.   , 0.982]), array([0.56, 0.  , 0.44]), array([0.6  , 0.002, 0.398]), array([0.788, 0.034, 0.178]), array([0.898, 0.004, 0.098]), array([0.6  , 0.002, 0.398]), array([0.83 , 0.004, 0.166]), array([0.65, 0.  , 0.35]), -20.354859993253363, 5.538748804609617, 6.174620619641426)


In [25]:
nobs = 500


a2,a1 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.01 & 0.32 & 0.73 & 0.77 & 0.89 & 0.66   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.05 & 0.00 & 0.00   \\
Model 2 & 0.99 & 0.68 & 0.27 & 0.18 & 0.11 & 0.34   \\
\hline
\end{tabular}
(array([0.012, 0.   , 0.988]), array([0.324, 0.   , 0.676]), array([0.726, 0.   , 0.274]), array([0.772, 0.046, 0.182]), array([0.91, 0.  , 0.09]), array([0.726, 0.   , 0.274]), array([0.888, 0.002, 0.11 ]), array([0.656, 0.   , 0.344]), -20.53801999641867, 5.848234129593286, 6.358875584315831)


In [26]:
nobs = 100


a2,a1 =np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.00 & 0.94 & 0.42 & 0.78 & 0.66 & 0.55   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.01 & 0.00 & 0.00   \\
Model 2 & 1.00 & 0.06 & 0.58 & 0.21 & 0.34 & 0.45   \\
\hline
\end{tabular}
(array([0.004, 0.   , 0.996]), array([0.942, 0.   , 0.058]), array([0.422, 0.   , 0.578]), array([0.776, 0.012, 0.212]), array([0.824, 0.004, 0.172]), array([0.422, 0.   , 0.578]), array([0.658, 0.002, 0.34 ]), array([0.548, 0.   , 0.452]), -20.964585553664627, 5.692687148134463, 5.796720428509175)


In [27]:
nobs = 250


a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=4)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.21 & 0.21 & 0.63 & 0.61 & 0.80 & 0.54   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.02 & 0.00 & 0.00   \\
Model 2 & 0.79 & 0.79 & 0.37 & 0.37 & 0.20 & 0.46   \\
\hline
\end{tabular}
(array([0.208, 0.   , 0.792]), array([0.212, 0.   , 0.788]), array([0.634, 0.   , 0.366]), array([0.61 , 0.016, 0.374]), array([0.686, 0.002, 0.312]), array([0.634, 0.   , 0.366]), array([0.8  , 0.004, 0.196]), array([0.54, 0.  , 0.46]), -12.562201428363693, 5.188241420246612, 4.9752441396435785)


In [28]:
nobs = 250


a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=19)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.01 & 0.54 & 0.64 & 0.82 & 0.86 & 0.65   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.03 & 0.00 & 0.00   \\
Model 2 & 0.99 & 0.46 & 0.36 & 0.15 & 0.13 & 0.35   \\
\hline
\end{tabular}
(array([0.014, 0.   , 0.986]), array([0.54, 0.  , 0.46]), array([0.644, 0.   , 0.356]), array([0.816, 0.03 , 0.154]), array([0.906, 0.002, 0.092]), array([0.644, 0.   , 0.356]), array([0.864, 0.002, 0.134]), array([0.648, 0.   , 0.352]), -20.193505130685438, 5.5835778298023175, 6.198470309470148)
